In [155]:
import pandas as pd
import numpy as np
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn import preprocessing, metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [156]:
#df1 = pd.read_csv("bellman_ford_performance_results.csv")
df1 = pd.read_csv("dijkstra_performance_results.csv")
conditions = [df1['vertexNum'] == 524288, df1['vertexNum'] == 1048576, df1['vertexNum'] == 2097152, df1['vertexNum'] == 4194304, df1['vertexNum'] == 8388608, df1['vertexNum'] == 16777216, df1['vertexNum'] == 33554432]
numbers = [1, 2, 4, 8, 16, 32, 64]
df1['graphSize'] = np.select(conditions, numbers, default=0)


In [157]:
#one-hot encoding - nothing is dropped

df1 = df1.drop(['vertexNum', 'edgeNum', 'maxEdgeDegree',
                'sAtomicMinBlock', 'sAtomicMaxBlock',
                'sAtomicAddBlock', 'sAtomicExchBlock'], axis=1)
df1.dropna()


,iterationNum,minProcessEdge,percentage,sOriginalDistance,sMinEdgetoProcess,sApprAttrValues,sReduceExecution,sPartialGraphProcess,Error,executionTime,graphSize
0,8,0,1.0,1,0,0,0,0,0.000000,42.250,1
1,8,2,1.0,0,1,0,0,0,0.153689,39.250,1
2,8,3,1.0,0,1,0,0,0,0.257669,39.875,1
3,8,4,1.0,0,1,0,0,0,0.379209,40.000,1
4,8,6,1.0,0,1,0,0,0,0.612974,39.625,1
...,...,...,...,...,...,...,...,...,...,...,...
144,8,99,1.0,0,1,0,0,0,3.419386,736.500,32
145,8,109,1.0,0,1,0,0,0,4.141858,724.000,32
146,8,115,1.0,0,1,0,0,0,4.903501,718.000,32
147,8,121,1.0,0,1,0,0,0,5.708293,716.125,32


In [158]:
#execution time prediction with minProcessEdge
"""
df1 = df1.drop(['vertexNum', 'edgeNum', 'maxEdgeDegree', 'percentage', 
                'sPartialGraphProcess', 'sApprAttrValues', 
                'sOriginalDistance', 'sReduceExecution','sMinEdgetoProcess', 
                'iterationNum', 'sAtomicMinBlock', 'sAtomicMaxBlock',
                'sAtomicAddBlock', 'sAtomicExchBlock'], axis=1)
df1.dropna()

"""



"\ndf1 = df1.drop(['vertexNum', 'edgeNum', 'maxEdgeDegree', 'percentage', \n                'sPartialGraphProcess', 'sApprAttrValues', \n                'sOriginalDistance', 'sReduceExecution','sMinEdgetoProcess', \n                'iterationNum', 'sAtomicMinBlock', 'sAtomicMaxBlock',\n                'sAtomicAddBlock', 'sAtomicExchBlock'], axis=1)\ndf1.dropna()\n\n"

In [159]:
df = df1[df1['minProcessEdge'] > 0]



for i in numbers:
    mini = df[df['graphSize'] == i]['minProcessEdge'].min()
    maxi = df[df['graphSize'] == i]['minProcessEdge'].max()
    #print(mini, maxi, i)
    df.loc[df['graphSize'] == i, ['minProcessEdge']] = 1.0 / (((df.loc[df['graphSize'] == i, ['minProcessEdge']] - mini) / (maxi - mini)) * 9 + 1)

    
    
print(df.head(50))

    iterationNum  minProcessEdge  percentage  sOriginalDistance  \
1              8        1.000000         1.0                  0   
2              8        0.979955         1.0                  0   
3              8        0.960699         1.0                  0   
4              8        0.924370         1.0                  0   
5              8        0.844530         1.0                  0   
6              8        0.765217         1.0                  0   
7              8        0.526316         1.0                  0   
8              8        0.515222         1.0                  0   
9              8        0.504587         1.0                  0   
10             8        0.489433         1.0                  0   
11             8        0.457380         1.0                  0   
12             8        0.273292         1.0                  0   
13             8        0.261593         1.0                  0   
14             8        0.250855         1.0                  

/home/buse/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [160]:
print(df1.head())

#df = df1.head(28)

   iterationNum  minProcessEdge  percentage  sOriginalDistance  \
0             8               0         1.0                  1   
1             8               2         1.0                  0   
2             8               3         1.0                  0   
3             8               4         1.0                  0   
4             8               6         1.0                  0   

   sMinEdgetoProcess  sApprAttrValues  sReduceExecution  sPartialGraphProcess  \
0                  0                0                 0                     0   
1                  1                0                 0                     0   
2                  1                0                 0                     0   
3                  1                0                 0                     0   
4                  1                0                 0                     0   

      Error  executionTime  graphSize  
0  0.000000         42.250          1  
1  0.153689         39.250          

In [161]:
#################### EXECUTION TIME #############

In [162]:
X = df.drop(['executionTime', 'Error'],axis=1)
y = df[['executionTime']]

In [163]:
X = X.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)


In [164]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#print(model.intercept_)

print(r2_score(y_pred, y_test))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.9940285123290733
Mean Absolute Error: 16.95128451742447
Mean Squared Error: 439.81242067815043
Root Mean Squared Error: 20.97170524011222


In [165]:
print(y_test)
print(y_pred)
#print(np.exp(y_test))
#print(np.exp(y_pred))

     executionTime
96         204.500
61         126.375
65         121.250
40          62.000
148        706.875
145        724.000
68         117.875
42          61.375
109        433.625
[[195.83623947]
 [118.53956371]
 [109.58860072]
 [ 57.1243867 ]
 [753.03120183]
 [753.70303613]
 [103.25347173]
 [ 49.48412992]
 [416.47128506]]


In [166]:
model = KNeighborsRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 8.030555555555544
Mean Squared Error: 141.4792361111107
Root Mean Squared Error: 11.894504450001719


In [167]:
print(y_test)
print(y_pred)

     executionTime
96         204.500
61         126.375
65         121.250
40          62.000
148        706.875
145        724.000
68         117.875
42          61.375
109        433.625
[[219.475]
 [124.975]
 [119.4  ]
 [ 62.925]
 [734.675]
 [734.675]
 [119.4  ]
 [ 62.125]
 [421.25 ]]


In [168]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 score: ', r2_score(y_test, y_pred))

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Absolute Error: 4.208333333333333
Mean Squared Error: 33.467013888888886
Root Mean Squared Error: 5.785068183598953
R2 score:  0.999479704115015
MAE: 3.465 (1.556)


In [169]:
print(y_test)
print(y_pred)

     executionTime
96         204.500
61         126.375
65         121.250
40          62.000
148        706.875
145        724.000
68         117.875
42          61.375
109        433.625
[215.625 126.375 122.25   62.5   716.125 718.    111.125  61.125 430.625]


In [170]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(x_train)
X_testscaled=sc_X.transform(x_test)
#X_trainscaled = x_train
#X_testscaled = x_test
#print(X_trainscaled)

In [171]:
#t = [(2),(3),(4),(5),(6),(7),(8),(9),(10),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(8,2)]

model = MLPRegressor(hidden_layer_sizes=(4,8,4),activation="relu" ,random_state=1, max_iter=50000)
model.fit(X_trainscaled, y_train.values.ravel()).loss_
print(model.loss_)
#print(model.loss_curve_)
print(model.n_outputs_)
print(model.best_loss_)
y_pred=model.predict(X_testscaled)


print(r2_score(y_pred, y_test))
print("Training set score: %f" % model.score(X_trainscaled, y_train.values.ravel()))
print("Test set score: %f" % model.score(X_testscaled, y_test))

106.1676142437075
1
106.16442100298283
0.9946015413033802
Training set score: 0.996290
Test set score: 0.993835


In [172]:
print(y_test)
print(y_pred)
print("Test set score: %f" % model.score(X_testscaled, y_test))

     executionTime
96         204.500
61         126.375
65         121.250
40          62.000
148        706.875
145        724.000
68         117.875
42          61.375
109        433.625
[193.91025835 118.99482894 108.74812624  50.52473566 750.34046702
 751.10955592 104.88887168  50.52473566 419.3429944 ]
Test set score: 0.993835


In [173]:
print("y1 MSE:%.4f" % mean_squared_error(y_test, y_pred)) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

#print("y2 MSE:%.4f" % mean_squared_error(y_test[:,1], y_pred[:,1]))

y1 MSE:396.5674
Mean Absolute Error: 16.737830221421802
Mean Squared Error: 396.56737624897136
Root Mean Squared Error: 19.913999504091873


In [174]:
################### ERROR ###################

In [175]:
y = df[['Error']]

In [176]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [177]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#print(model.intercept_)

print(r2_score(y_pred, y_test))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.4925876934411514
Mean Absolute Error: 2.2013468008985044
Mean Squared Error: 6.66822049449197
Root Mean Squared Error: 2.5822897774053106


In [178]:
print(y_test)
print(y_pred)

         Error
96   15.382136
61    3.751060
65    6.602716
40    7.774907
148   6.567028
145   4.141858
68   11.813880
42   13.174481
109   0.114736
[[10.45967039]
 [ 5.96999832]
 [ 8.86606315]
 [ 7.21613115]
 [ 7.17351028]
 [ 6.95613974]
 [10.91578051]
 [ 9.6881198 ]
 [-1.92863358]]


In [179]:
model = KNeighborsRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 2.5548012
Mean Squared Error: 11.826724330617255
Root Mean Squared Error: 3.439000484242079


In [180]:
print(y_test)
print(y_pred)

         Error
96   15.382136
61    3.751060
65    6.602716
40    7.774907
148   6.567028
145   4.141858
68   11.813880
42   13.174481
109   0.114736
[[7.2438438]
 [3.9032956]
 [8.6336026]
 [5.7400734]
 [3.9811166]
 [3.9811166]
 [8.6336026]
 [9.4050122]
 [1.0552998]]


In [181]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 score: ', r2_score(y_test, y_pred))

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Absolute Error: 1.6201328888888886
Mean Squared Error: 6.265611098687553
Root Mean Squared Error: 2.503120272517394
R2 score:  0.7096163671036437
MAE: 0.873 (0.502)


In [182]:
print(y_test)
print(y_pred)

         Error
96   15.382136
61    3.751060
65    6.602716
40    7.774907
148   6.567028
145   4.141858
68   11.813880
42   13.174481
109   0.114736
[16.265274  4.457448  7.922143  7.01748  13.39053   4.903501 13.731627
 11.767733  0.119912]


In [183]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(x_train)
X_testscaled=sc_X.transform(x_test)

In [184]:
model = MLPRegressor(hidden_layer_sizes=(4,8,4),activation="relu" ,random_state=1, max_iter=50000)
model.fit(X_trainscaled, y_train.values.ravel()).loss_
print(model.loss_)
#print(model.loss_curve_)
print(model.n_outputs_)
print(model.best_loss_)
y_pred=model.predict(X_testscaled)


print(r2_score(y_pred, y_test))
print("Training set score: %f" % model.score(X_trainscaled, y_train.values.ravel()))
print("Test set score: %f" % model.score(X_testscaled, y_test))

0.5280936507544335
1
0.5280936507544335
0.8482526859560977
Training set score: 0.942792
Test set score: 0.813945


In [185]:
print(y_test)
print(y_pred)
print("Test set score: %f" % model.score(X_testscaled, y_test))

         Error
96   15.382136
61    3.751060
65    6.602716
40    7.774907
148   6.567028
145   4.141858
68   11.813880
42   13.174481
109   0.114736
[15.90218098  4.86480407  8.44902869  7.36527549  6.17863753  5.48835948
 17.13468839 12.35406593  0.42311384]
Test set score: 0.813945


In [186]:
print("y1 MSE:%.4f" % mean_squared_error(y_test, y_pred)) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

y1 MSE:4.0145
Mean Absolute Error: 1.3415807221698886
Mean Squared Error: 4.01451491680522
Root Mean Squared Error: 2.0036254432416305
